#Vanilla RNN - Sentiment Analysis - FIFA World Cup 2022 (con GridSearchCV en Colab)

In [1]:
!pip install pandas==2.2.2
!pip install numpy==1.26.4
!pip install matplotlib==3.8.4
!pip install seaborn==0.13.2
!pip install nltk==3.8.1
!pip install keras==3.10.0
!pip install scikeras==0.13.0
!pip install jupyterlab==4.1.5
!pip install notebook==7.1.3
!pip install scikit-learn==1.3.2
!pip install tensorflow==2.18.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 878.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 16.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 88.9 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.0
    Uninstalling matplotlib-3.10.0:
      Successfully uninstalled matplotlib-3.10.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.9 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.9.1
    Uninstalling nltk-3.9.1:
      Successfully uninstalled nltk-3.9.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
textblob 0.19.0 requires nltk>=3.9, but you have nltk 3.8.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.4 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.8.0
    Uninstalling keras-3.8.0:
      Successfully uninstalled keras-3.8.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 4

In [1]:
import pandas as pd
from google.colab import files
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout


## 1. Subir y cargar el CSV




In [2]:
uploaded = files.upload()
filename = next(iter(uploaded))
df = pd.read_csv(filename)
df.head()


Saving fifa_tweets_clean.csv to fifa_tweets_clean.csv


,Tweet,Sentiment,test_clean,processed_tokens,sentiment_label
0,What are we drinking today @TucanTribe \r\n@Ma...,neutral,what are we drinking today _mention_tucantribe...,"['drinking', 'today', '_mention_tucantribe_', ...",1
1,Amazing @CanadaSoccerEN #WorldCup2022 launch ...,positive,amazing _mention_canadasocceren_ _hashtag_worl...,"['amazing', '_mention_canadasocceren_', '_hash...",2
2,Worth reading while watching #WorldCup2022 htt...,positive,worth reading while watching _hashtag_worldcup...,"['worth', 'reading', 'watching', '_hashtag_wor...",2
3,Golden Maknae shinning bright\r\n\r\nhttps://t...,positive,golden maknae shinning bright _hashtag_jeonjun...,"['golden', 'maknae', 'shinning', 'bright', '_h...",2
4,"If the BBC cares so much about human rights, h...",negative,if the bbc cares so much about human rights ho...,"['bbc', 'cares', 'much', 'human', 'rights', 'h...",0


## 2. Preprocesamiento

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, cohen_kappa_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout, Bidirectional
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, cohen_kappa_score

# --- 1. Preparación de datos ---
# Unir tokens en texto plano
texts = df["processed_tokens"].apply(lambda tokens: " ".join(eval(tokens)))
labels = df["sentiment_label"]

# Tokenización
vocab_size = 15000
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

# --- Datos ---
# Asegúrate de tener ya listos: padded_sequences, labels
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# --- Class weights ---
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

# --- Modelo SOLO RNN ---
model = Sequential([
    Embedding(input_dim=15000, output_dim=64),
    Bidirectional(SimpleRNN(128, return_sequences=True)),
    Dropout(0.3),
    SimpleRNN(64),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# --- Entrenamiento ---
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=64,
    validation_split=0.2,
    class_weight=class_weights_dict
)

# --- Evaluación ---
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['negative', 'neutral', 'positive']))
print("Cohen Kappa Score:", cohen_kappa_score(y_test, y_pred))


Epoch 1/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 52s 202ms/step - accuracy: 0.4821 - loss: 0.9745 - val_accuracy: 0.7238 - val_loss: 0.6332
Epoch 2/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 50s 198ms/step - accuracy: 0.7553 - loss: 0.5812 - val_accuracy: 0.7411 - val_loss: 0.5980
Epoch 3/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 49s 195ms/step - accuracy: 0.8890 - loss: 0.2889 - val_accuracy: 0.7542 - val_loss: 0.7010
Epoch 4/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 48s 193ms/step - accuracy: 0.9124 - loss: 0.2299 - val_accuracy: 0.7402 - val_loss: 0.8287
Epoch 5/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 48s 192ms/step - accuracy: 0.9331 - loss: 0.1815 - val_accuracy: 0.7372 - val_loss: 0.9123
Epoch 6/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 47s 191ms/step - accuracy: 0.9546 - loss: 0.1287 - val_accuracy: 0.7351 - val_loss: 1.0913
Epoch 7/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 47s 190ms/step - accuracy: 0.9582 - loss: 0.1130 - val_accuracy: 0.7500 - val_loss: 1.2142
Epoch 8/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 46s 189ms/step - accuracy: 0.9663 - loss: 0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
